In [1]:
import numpy as np
import pandas
import pandas as pd

In [2]:
import rbot


In [3]:
ftx = rbot.FtxMarket("BTC-PERP")

In [4]:
session = rbot._DummySession()

In [5]:
session.center_price

0.0

In [6]:
session.current_timestamp

0

In [7]:
session.run()

In [8]:
session.current_timestamp

3

In [9]:
session.long_order_size

0.0

In [10]:
session.make_order(1, rbot.OrderSide.Buy, 10.0, 10.0, 10, "")

OrderStatus.InOrder

In [11]:
session.long_orders[0]

{order_index:0, create_time:1, order_id:0000-0001, order_side:Buy, post_only:true, valid_until:10000003, price:10, size:10, message:, remain_size10}

In [12]:
session.long_order_size

10.0

In [13]:
class Market:
    MARKET = {}
    DUMMY_MODE = True

    @classmethod
    def dummy_mode(cls, dummy=True):
        cls.DUMMY_MODE = dummy

    @classmethod
    def open(cls, exchange: str, market):
        exchange = exchange.upper()
        if exchange == "FTX":
            m = rbot.FtxMarket(market, cls.DUMMY_MODE)
            cls.MARKET[exchange.upper() + "/" + market.upper()] = m
            return m
        

    @classmethod
    def download(cls, ndays):
        for m in cls.MARKET:
            cls.MARKET[m].download(ndays)
    
    @classmethod
    def get(cls, exchange, market):
        return cls.MARKET[exchange.upper() + "/" + market.upper()]
            

In [14]:
rbot.init_log()
Market.dummy_mode()
Market.open("FTX", "BTC-PERP")
Market.download(1)
Market.get("FTX", "BTC-PERP")

2022-11-09T12:43:36.383Z DEBUG [rbot::db::df] Select from 1970-01-01T00:00:00.000000 -> 1970-01-01T00:00:00.000000
2022-11-09T12:43:36.383Z DEBUG [rbot::db::df] preserve select df
2022-11-09T12:43:36.384Z DEBUG [rbot::exchange::ftx] Diff donwload for 1 days
2022-11-09T12:43:37.770Z DEBUG [rbot::db::sqlite] 2022-11-08T12:43:36.502722- gap1667911416502722
2022-11-09T12:43:37.774Z DEBUG [rbot::db::sqlite] after db "2022-11-09T12:26:03.803126"-"2022-11-09T12:43:36.384891"
2022-11-09T12:43:37.774Z DEBUG [rbot::exchange::ftx] [TimeChunk { start: 1667996763803126, end: 1667997816384891 }]
2022-11-09T12:43:37.774Z DEBUG [rbot::exchange::ftx::rest] download trade call back start_time 2022-11-09T12:26:03.803126 -> end_time 2022-11-09T12:43:36.384891
2022-11-09T12:43:37.774Z DEBUG [rbot::exchange::ftx::rest] download trade to: 2022-11-09T12:26:03.803126-2022-11-09T12:43:36.384891(1667997816384891)
2022-11-09T12:43:37.774Z DEBUG [rbot::exchange::ftx::rest] https://ftx.com/api/markets/BTC-PERP/trad

In [15]:
ftx = Market.get("FTX", "BTC-PERP")

In [16]:
ftx.info()

'{"start": 2022-09-04T09:51:40.853933, "end": 2022-11-09T12:43:36.740105, "count": 29173460}'

In [17]:
class Session:
    def __init__(self, market, agent):
        self.session = rbot._DummySession()
        self.market = market
        self.agent = agent

        if hasattr(self.agent, 'clock_interval'):
            self.clock_interval = self.agent.clock_interval()
        else:
            print("Agent class has no clock_interval() sttings, use default 60[sec]");            
            self.clock_interval = 60

        
    def __getattr__(self, func):
        return getattr(self.session, func)

                                       
    def want_on_tick(self):
        return hasattr(self.agent, 'on_tick')

    def want_on_clock(self):
        return hasattr(self.agent, 'on_clock')
    
    def want_on_update(self):
        return hasattr(self.agent, 'on_update')
    
    def make_orders(self, orders):
        for order in orders:
            self.make_order(order)
    
    def make_order(self, order):
        self.session.make_order(order)
    
    def on_tick(self, time, price, side, size):
        return self.agent.on_tick(self, time, price, side, size)
    
    def on_clock(self):
        return self.agent.on_clock(self, time)
        
    def on_update(self, time, order_result):
        return self.agent.on_update(self, time, order_result)
        
    def ohlcvv(self, time_window, num_of_bars, exchange_name=None, market_name=None):
        if market_name == None:
            market = self.market
        else:
            market = Market.get(exchange_name, market_name)

        now = self.session.current_timestamp
        return market.ohlcvv(now - time_window * num_of_bars * 1_000_000, now, time_window)

        

In [18]:
class Agent:
    def clock_interval(self):
        return 60    # Sec
    
    def on_tick(self, session: Session, time, price, side, size):
        print("tick", session.current_timestamp, time, price, side, size)

In [19]:
s = Session(ftx, Agent())

In [20]:
s.want_on_tick()

True

In [21]:
s.on_tick(1, 1, "buy", 10.0)

tick 0 1 1 buy 10.0


In [22]:
ohlcv = s.ohlcvv(60, 3)

2022-11-09T12:44:54.676Z DEBUG [rbot::db::sqlite] cache update all 1969-12-31T23:57:00.000000 -> 1970-01-01T00:00:00.000000
2022-11-09T12:44:54.676Z DEBUG [rbot::db::sqlite] create iter 0 microsec
2022-11-09T12:45:19.434Z DEBUG [rbot::db::df] Select from 1969-12-31T23:57:00.000000 -> 1970-01-01T00:00:00.000000
2022-11-09T12:45:19.819Z DEBUG [rbot::db::df] ohlc "1969-12-31T23:57:00.000000" -> "1970-01-01T00:00:00.000000"
2022-11-09T12:45:19.819Z DEBUG [rbot::db::df] Select from 1969-12-31T23:57:00.000000 -> 1970-01-01T00:00:00.000000


In [28]:
ohlcv2 = Market.get("FTX", "BTC-PERP").ohlcvv(rbot.NOW()- 1_000_000*60*10, rbot.NOW(), 10)

2022-11-09T12:47:19.954Z DEBUG [rbot::db::sqlite] create iter 1 microsec
2022-11-09T12:47:19.955Z DEBUG [rbot::db::sqlite] load data AFTER cache df1=(29173460, 5) df2=(2, 5)
2022-11-09T12:47:19.955Z DEBUG [rbot::db::df] Select from 1970-01-01T00:00:00.000000 -> 2022-11-09T12:43:36.740105
2022-11-09T12:47:20.107Z DEBUG [rbot::db::sqlite] cache update diff after 2022-11-09T12:37:00.000000 -> 2022-11-09T12:47:00.000000
2022-11-09T12:47:20.107Z DEBUG [rbot::db::df] Select from 1970-01-01T00:00:00.000000 -> 2022-11-09T12:37:00.000000
2022-11-09T12:47:20.107Z DEBUG [rbot::db::df] Select from 2022-11-09T12:37:00.000000 -> 2022-11-09T12:47:00.000000
2022-11-09T12:47:20.203Z DEBUG [rbot::db::df] Select from 1970-01-01T00:00:00.000000 -> 2022-11-09T12:37:00.000000
2022-11-09T12:47:20.204Z DEBUG [rbot::db::df] Select from 2022-11-09T12:37:00.000000 -> 2022-11-09T12:47:19.931429


In [26]:
ohlcv2 = Market.get("FTX", "BTC-PERP").ohlcvv(- 1_000_000*60*10, 1, 10)


2022-11-09T12:46:54.471Z DEBUG [rbot::db::sqlite] create iter 8 microsec
2022-11-09T12:46:54.472Z DEBUG [rbot::db::sqlite] load data before cache df1=(0, 5) df2=(29173460, 5)
2022-11-09T12:46:54.479Z DEBUG [rbot::db::df] Select from 1970-01-01T00:00:00.000000 -> 2022-09-04T09:51:40.853933
2022-11-09T12:46:54.479Z DEBUG [rbot::db::sqlite] cache update diff before 1969-12-31T23:50:00.000000 -> 2022-09-04T09:51:00.000000
2022-11-09T12:46:54.479Z DEBUG [rbot::db::df] Select from 1969-12-31T23:50:00.000000 -> 2022-09-04T09:51:00.000000
2022-11-09T12:46:54.620Z DEBUG [rbot::db::df] Select from 2022-09-04T09:51:00.000000 -> 1970-01-01T00:00:00.000000
2022-11-09T12:46:54.621Z DEBUG [rbot::db::df] Select from 1970-01-01T00:00:00.000000 -> 2022-09-04T09:51:00.000000
2022-11-09T12:46:54.621Z DEBUG [rbot::db::df] Select from 1969-12-31T23:50:00.000000 -> 1970-01-01T00:00:00.000001


In [29]:
ohlcv2

,order_side,open,high,low,close,vol,count
timestamp,,,,,,,
2022-11-09 12:37:00+00:00,Buy,18130.0,18141.0,18117.0,18121.0,11.0998,51.0
2022-11-09 12:37:00+00:00,Sell,18136.0,18140.0,18117.0,18117.0,19.5994,60.0
2022-11-09 12:37:10+00:00,Buy,18125.0,18128.0,18120.0,18128.0,2.2613,32.0
2022-11-09 12:37:10+00:00,Sell,18126.0,18126.0,18118.0,18126.0,8.2731,31.0
2022-11-09 12:37:20+00:00,Buy,18132.0,18151.0,18130.0,18149.0,13.4190,33.0
...,...,...,...,...,...,...,...
2022-11-09 12:43:10+00:00,Sell,18206.0,18216.0,18192.0,18209.0,15.9797,71.0
2022-11-09 12:43:20+00:00,Buy,18213.0,18222.0,18213.0,18216.0,10.8137,48.0
2022-11-09 12:43:20+00:00,Sell,18214.0,18222.0,18214.0,18216.0,5.5584,21.0


In [14]:
s = rbot.Session()

In [16]:
s

In [19]:
s2 = rbot._DummySession()

In [20]:
s2.get_center_price()

0.0

In [47]:
df2 = pd.read_csv('ftx.csv', names=("timestamp", 'price', 'order_side', 'liquid', 'id'), i)

In [48]:
df2

,timestamp,price,order_side,liquid,id
1667314934485667,20391,0.0001,Sell,false,5283994328
1667314934380211,20391,0.0001,Sell,false,5283994317
1667314934341958,20392,0.2453,Buy,false,5283994311
1667314934286755,20392,0.0260,Buy,false,5283994301
1667314934271012,20391,0.0001,Sell,false,5283994291
...,...,...,...,...,...
1667227934951002,20350,0.0100,Buy,false,5274339831
1667227934951002,20350,0.0010,Buy,false,5274339830
1667227934439297,20349,0.0010,Sell,false,5274339780
1667227934285876,20349,0.0015,Buy,false,5274339748


In [39]:
df2['diff'] = df2['timestamp'].diff()

In [40]:
df2['diff']

1667314934485667    NaN
1667314934380211    0.0
1667314934341958    1.0
1667314934286755    0.0
1667314934271012   -1.0
                   ... 
1667227934951002    0.0
1667227934951002    0.0
1667227934439297   -1.0
1667227934285876    0.0
1667227934285876    0.0
Name: diff, Length: 293968, dtype: float64

In [45]:
df2[df2['diff'] < -1]

,timestamp,price,order_side,liquid,id,diff
1667314810143806,20405,0.0001,Sell,false,5283979059,-2.0
1667314729898654,20388,0.0011,Buy,false,5283972144,-2.0
1667314723949493,20389,0.1000,Buy,false,5283971641,-2.0
1667314723930986,20388,0.2390,Buy,false,5283971630,-3.0
1667314721484732,20383,0.0200,Buy,false,5283971168,-2.0
...,...,...,...,...,...,...
1667227982525311,20348,1.0762,Buy,false,5274349198,-2.0
1667227981683112,20345,0.0010,Buy,false,5274349097,-2.0
1667227961829144,20346,0.0010,Buy,false,5274345177,-2.0
1667227960847934,20344,1.6144,Buy,false,5274344978,-2.0
